In [2]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, after_nearest_workday, next_workday
from pandas.tseries.holiday import *
import pandas.tseries.holiday
from pandas.tseries.offsets import *
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Task 1

In [416]:
data = pd.read_csv('./business.csv', encoding='windows-1251', parse_dates=True, sep =',')

In [440]:
#Holiday declaration
first_Jan_1994 = Holiday(
    'New_Year',
    start_date = datetime.datetime(1999, 1, 1),
    end_date = datetime.datetime(2002,1,1),
    month = 1,
    day = 1,
    observance=next_monday)
second_Jan_1994 = Holiday(
    'New_Year',
    start_date = datetime.datetime(1999, 1, 1),
    end_date = datetime.datetime(2002,1,1),
    month = 1,
    day = 2,
    observance=next_monday_or_tuesday)
Christmas_1994 = Holiday(
    'Christmas',
    start_date = datetime.datetime(1999, 1, 1),
    end_date = datetime.datetime(2025,1,1),
    month = 1,
    day = 7,
    observance=next_monday)
first_may_1994 = Holiday(
    'Holiday of spring and labour',
    start_date = datetime.datetime(1999, 1, 1),
    #end_date = datetime.datetime(2005,1,1),
    month = 5,
    day = 1,
    observance=next_monday)
second_may_1994 = Holiday(
    'Holiday of spring and labour',
    start_date = datetime.datetime(1999, 1, 1),
    end_date = datetime.datetime(2005,1,1),
    month = 5,
    day = 2,
    observance=next_monday_or_tuesday)
vic_day = Holiday(
    'Victory Day',
    start_date = datetime.datetime(1999, 1, 1),
    end_date = datetime.datetime(2005,1,1),
    month = 5,
    day = 9,
    observance=next_monday)
day_of_Russia = Holiday(
    'Day of Russia',
    start_date = datetime.datetime(1999, 1, 1),
    #end_date = datetime.datetime(2025,,1),
    month = 6,
    day = 12,
    observance=next_monday)
Revolution = Holiday(
    'Great Revolution Day',
    start_date = datetime.datetime(1999, 1, 1),
    end_date = datetime.datetime(2005,1,1),
    month = 11,
    day = 7,
    observance=next_monday)
constit = Holiday(
    'Day of Constitution',
    start_date = datetime.datetime(1999, 1, 1),
    end_date = datetime.datetime(2004,12,31),
    month = 12 ,
    day = 12,
    observance=next_monday)
defender = Holiday(
    'Day of Motherlands Defender',
    start_date = datetime.datetime(2002, 2, 1),
    end_date = datetime.datetime(2025,12,31),
    month = 2 ,
    day = 23,
    observance=next_monday)
unity = Holiday(
    'Day of People\'s unity',
    start_date = datetime.datetime(2005, 1, 1),
    end_date = datetime.datetime(2025,12,31),
    month = 11 ,
    day = 4,
    observance=next_monday)
march_8 = Holiday('March 8th', month=3, day=8, observance=nearest_workday)

In [453]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
    week_mask = ['Mon Tue Wed Thu Fri']
    rules = [
             first_Jan_1994,
             second_Jan_1994,
             Christmas_1994,
             march_8,
             first_may_1994,
            second_may_1994,
            vic_day,
            day_of_Russia,
            Revolution,
            constit,
            defender,
            unity] + [Holiday('New Year', start_date = datetime.datetime(2005, 1, 1), end_date = datetime.datetime(2025,12,31), observance = None, day = i, month = 1) 
                      for i in range(1, 6)] + [Holiday('New Year', start_date = datetime.datetime(2012, 4, 26), end_date = datetime.datetime(2025,12,31), observance = next_monday, day = i, month = 1) 
                      for i in [6,8]]


In [425]:
rus_month = ['январь', 'февраль', 'март', "апрель", "май", "июнь", "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь"]

In [510]:
Russia_BD = CustomBusinessDay(calendar=RussianBusinessCalendar())
rus_day_offset = pd.offsets.CustomBusinessDay(calendar=RussianBusinessCalendar())
rus_month_offset = pd.offsets.CustomBusinessMonthEnd(calendar=RussianBusinessCalendar())
s = pd.date_range('2014-01-01', end='2015-01-5', freq='d')
df = pd.DataFrame(index = s, columns=['выходной', 'праздник', 'предпраздничный рабочий день', 'последний день месяца', 'последний рабочий день месяца', 'пн', 'вт', 'ср',
                                       'чт', 'пт', 'сб', 'вс'] + rus_month + ['предновогодний выходной день'])

In [511]:
holidays = RussianBusinessCalendar().holidays(start=s.min(), end=s.max())
bef_newyear = pd.date_range(df.index[0], df.index[df.index.size - 1], freq = 'Y')

In [512]:
for i in range (df.index.size):
    df.iloc[i,3] = df.index[i].is_month_end
    df.iloc[i,0] = df.index[i].dayofweek == 5 or df.index[i].dayofweek == 6 or df.index[i] in holidays
    df.iloc[i,1] = df.index[i] in holidays
    df.iloc[i, 4] = False
    df.iloc[i, 2] = False
    df.iloc[i, 24] = False
    if rus_month_offset.rollforward(df.index[i]) in df.index:
        df.loc[rus_month_offset.rollforward(df.index[i])][4] = True
        
    for j in range(5,13):
        df.iloc[i,j] = df.index[i].dayofweek == (j - 5)
    for j in range(12,25):
        df.iloc[i,j] = df.index[i].month == (j - 11)
    
        
for hil in holidays:
    if rus_day_offset.rollback(hil) in df.index:   
        df.loc[rus_day_offset.rollback(hil)][2] = True
        
for wk in bef_newyear:
    if (wk - pd.Timedelta(days = wk.dayofweek + 1)) in df.index:   
        df.loc[wk - pd.Timedelta(days = wk.dayofweek + 1)][24] = True


In [519]:
df = df.astype(int)
df

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,...,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-01,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-02,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2015-01-03,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-04,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Task 2

In [823]:
ts = pd.read_csv('./sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], 
                 index_col='TRADEDATE', parse_dates=True)
ts = ts.CLOSE

In [824]:
cal_ts = ts.asfreq(freq = rus_day_offset).fillna(-1)

In [837]:
rolling= pd.DataFrame( index = cal_ts.index)
rolling['Data'] = cal_ts
rolling['Weekday'] = rolling.index.weekday

In [838]:
for i in [1, 3,5]:
    rolling['Lagged_{}'.format(i)] = rolling['Data'].shift(i)
    rolling['Rolled_win_{}_mean'.format(i)] = rolling.groupby('Weekday')['Data'].transform(lambda x: x.rolling(window=i).mean())
    rolling['Rolled_win_{}_median'.format(i)] = rolling.groupby('Weekday')['Data'].transform(lambda x: x.rolling(window=i).median())
    rolling['Rolled_win_{}_max'.format(i)] = rolling.groupby('Weekday')['Data'].transform(lambda x: x.rolling(window=i).max())
    rolling['Rolled_win_{}_min'.format(i)] = rolling.groupby('Weekday')['Data'].transform(lambda x: x.rolling(window=i).min())
    rolling['Rolled_win_{}_std'.format(i)] = rolling.groupby('Weekday')['Data'].transform(lambda x: x.rolling(window=i).std())
    
   

In [944]:
enroll = pd.DataFrame( index = cal_ts.index)
enroll['Data'] = cal_ts
enroll['Weekday'] = enroll.index.weekday
enroll['Holiday'] = False
enroll['Before Holiday'] = False
enroll['Business'] = False

In [945]:
holidays = RussianBusinessCalendar().holidays(start=enroll.index.min(), end=enroll.index.max())
bus_day = pd.date_range(start = enroll.index[0], end=enroll.index[enroll.index.size - 1 ], freq = rus_day_offset)

In [946]:
for i in range (enroll.index.size):
    holi_enroll = enroll.index[i] in holidays
    business =  enroll.index[i] in bus_day
     
for hil in holidays:
    if rus_day_offset.rollback(hil) in enroll.index:   
        enroll.loc[rus_day_offset.rollback(hil), 'Before Holiday'] = True

In [949]:
enroll['Business'] = business
enroll['Holiday'] = holi_enroll

In [950]:
enroll   #Банк, к примеру, не работал 31 декабря, хотя официально это рабочий день - так что рабочий для кого - непонятно

,Data,Weekday,Holiday,Before Holiday,Business
TRADEDATE,,,,,
2013-03-25,98.79,0,False,False,True
2013-03-26,97.20,1,False,False,True
2013-03-27,96.75,2,False,False,True
2013-03-28,98.59,3,False,False,True
2013-03-29,98.76,4,False,False,True
...,...,...,...,...,...
2019-12-27,252.06,4,False,False,True
2019-12-30,254.75,0,False,False,True
2019-12-31,-1.00,1,False,True,True


In [951]:
for str in ['Business', 'Holiday', 'Before Holiday']:
    enroll[str] = enroll[str].astype(int)
    enroll[str + '_mean'] = enroll.groupby(str)['Data'].transform(lambda x: x.expanding().mean())
    enroll[str + '_median'] = enroll.groupby(str)['Data'].transform(lambda x: x.expanding().median())
    enroll[str + '_max'] = enroll.groupby(str)['Data'].transform(lambda x: x.expanding().max())
    enroll[str + '_min'] = enroll.groupby(str)['Data'].transform(lambda x: x.expanding().min())
    enroll[str + '_std'] = enroll.groupby(str)['Data'].transform(lambda x: x.expanding().std())
    
enroll

,Data,Weekday,Holiday,Before Holiday,Business,Business_mean,Business_median,Business_max,Business_min,Business_std,Holiday_mean,Holiday_median,Holiday_max,Holiday_min,Holiday_std,Before Holiday_mean,Before Holiday_median,Before Holiday_max,Before Holiday_min,Before Holiday_std
TRADEDATE,,,,,,,,,,,,,,,,,,,,
2013-03-25,98.79,0,0,0,1,98.790000,98.790,98.79,98.79,NaN,98.790000,98.790,98.79,98.79,NaN,98.790000,98.790,98.79,98.79,NaN
2013-03-26,97.20,1,0,0,1,97.995000,97.995,98.79,97.20,1.124300,97.995000,97.995,98.79,97.20,1.124300,97.995000,97.995,98.79,97.20,1.124300
2013-03-27,96.75,2,0,0,1,97.580000,97.200,98.79,96.75,1.071774,97.580000,97.200,98.79,96.75,1.071774,97.580000,97.200,98.79,96.75,1.071774
2013-03-28,98.59,3,0,0,1,97.832500,97.895,98.79,96.75,1.010359,97.832500,97.895,98.79,96.75,1.010359,97.832500,97.895,98.79,96.75,1.010359
2013-03-29,98.76,4,0,0,1,98.018000,98.590,98.79,96.75,0.968334,98.018000,98.590,98.79,96.75,0.968334,98.018000,98.590,98.79,96.75,0.968334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,252.06,4,0,0,1,144.237488,139.075,279.97,-1.00,63.228509,144.237488,139.075,279.97,-1.00,63.228509,144.473283,139.200,279.97,-1.00,62.896555
2019-12-30,254.75,0,0,0,1,144.302610,139.150,279.97,-1.00,63.266768,144.302610,139.150,279.97,-1.00,63.266768,144.539795,139.295,279.97,-1.00,62.935871
2019-12-31,-1.00,1,0,1,1,144.217038,139.075,279.97,-1.00,63.346343,144.217038,139.075,279.97,-1.00,63.346343,130.838750,116.510,277.49,-1.00,78.294255


In [952]:
pd.concat([rolling,enroll], axis=1) #final concatenated

,Data,Weekday,Lagged_1,Rolled_win_1_mean,Rolled_win_1_median,Rolled_win_1_max,Rolled_win_1_min,Rolled_win_1_std,Lagged_3,Rolled_win_3_mean,...,Holiday_mean,Holiday_median,Holiday_max,Holiday_min,Holiday_std,Before Holiday_mean,Before Holiday_median,Before Holiday_max,Before Holiday_min,Before Holiday_std
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2013-03-25,98.79,0,NaN,98.79,98.79,98.79,98.79,NaN,NaN,NaN,...,98.790000,98.790,98.79,98.79,NaN,98.790000,98.790,98.79,98.79,NaN
2013-03-26,97.20,1,98.79,97.20,97.20,97.20,97.20,NaN,NaN,NaN,...,97.995000,97.995,98.79,97.20,1.124300,97.995000,97.995,98.79,97.20,1.124300
2013-03-27,96.75,2,97.20,96.75,96.75,96.75,96.75,NaN,NaN,NaN,...,97.580000,97.200,98.79,96.75,1.071774,97.580000,97.200,98.79,96.75,1.071774
2013-03-28,98.59,3,96.75,98.59,98.59,98.59,98.59,NaN,98.79,NaN,...,97.832500,97.895,98.79,96.75,1.010359,97.832500,97.895,98.79,96.75,1.010359
2013-03-29,98.76,4,98.59,98.76,98.76,98.76,98.76,NaN,97.20,NaN,...,98.018000,98.590,98.79,96.75,0.968334,98.018000,98.590,98.79,96.75,0.968334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,252.06,4,248.24,252.06,252.06,252.06,252.06,NaN,248.67,245.993333,...,144.237488,139.075,279.97,-1.00,63.228509,144.473283,139.200,279.97,-1.00,62.896555
2019-12-30,254.75,0,252.06,254.75,254.75,254.75,254.75,NaN,248.04,248.333333,...,144.302610,139.150,279.97,-1.00,63.266768,144.539795,139.295,279.97,-1.00,62.935871
2019-12-31,-1.00,1,254.75,-1.00,-1.00,-1.00,-1.00,NaN,248.24,163.090000,...,144.217038,139.075,279.97,-1.00,63.346343,130.838750,116.510,277.49,-1.00,78.294255
